<h1>CHAPTER 12 - Inheritance: for good or for worse</h1>

• The pitfalls of subclassing from built-in types. <br>
 • Multiple inheritance and the method resolution order.

In [26]:
import sys
print(sys.executable)

/home/pyodide/this.program


In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
       super().__setitem__(key, [value] * 2) 

DoppelDict.__setitem__ duplicates values when storing (for no good reason,
 just to have a visible effect). It works by delegating to the superclass.

The __init__ method inherited from dict clearly ignored that __setitem__
 was overridden: the value of 'one' is not duplicated.

In [2]:
dd = DoppelDict(one=1)

In [3]:
dd

{'one': 1}

The [] operator calls our __setitem__ and works as expected: 'two' maps to
 the duplicated value [2, 2].

In [5]:
dd['two'] = 2 
dd

{'one': 1, 'two': [2, 2]}

The update method from
dict does not use our version of __setitem__ either:
 the value of 'three' was not duplicated.

In [6]:
dd.update(three=3)  

In [7]:
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [8]:
class AnswerDict(dict):
    def __getitem__(self, key):  # 
        return 42

In [10]:
ad = AnswerDict(a='foo') 
ad['a'] 

42

 d is an instance of plain dict, which we update with ad.

In [11]:
d = {}
d.update(ad) 
d['a']

'foo'

In [12]:
 d

{'a': 'foo'}

DoppelDict2 and AnswerDict2 work as expected because they extend
 UserDict and not dict

In [13]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [14]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
print(ad['a'])
d = {}
d.update(ad)
print(d['a'])
print(d)

42
42
{'a': 42}


collections.UserDict is a wrapper class around a regular Python dictionary. Instead of inheriting directly from the C-implemented dict, your custom dictionary class inherits from UserDict.

DoppelDict2(collections.UserDict):

When DoppelDict2(one=1) is called, the __init__ of UserDict (or potentially overridden in DoppelDict2 if you did so) will eventually use the item setting mechanism, which will call your overridden __setitem__, resulting in {'one': [1, 1]}.
dd2['two'] = 2 directly calls your overridden __setitem__.
dd2.update(three=3) uses the update method of UserDict, which internally uses the item setting mechanism, again calling your __setitem__.
AnswerDict2(collections.UserDict):

When ad2['a'] is called, the __getitem__ method of UserDict (or your override) is invoked, which in this case returns 42.
d.update(ad2) uses the update method of the regular dict d. However, when update iterates through ad2 to get key-value pairs, it will call the __getitem__ of ad2 to retrieve the values, thus getting 42 for all keys.

<h2> Multiple inheritance and method resolution order</h2>

<h3>diamond.py: classes A, B, C and D</h3>

In [15]:
class A:
    def ping(self):
        print('ping:', self)
class B(A):
    def pong(self):
        print('pong:', self)
class C(A):
    def pong(self):
        print('PONG:', self)


In [17]:
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [19]:
d = D()
d.pong()

pong: <__main__.D object at 0x2787c50>


In [20]:
C.pong(d)  

PONG: <__main__.D object at 0x2787c50>


The __mro__ Attribute: Every class in Python has a special attribute called __mro__ (Method Resolution Order). This attribute holds a tuple that lists the class itself and its superclasses in the order they are searched.

MRO for Class D: The text shows the __mro__ for class D (though the exact output might vary slightly depending on the Python version):

In [22]:
 D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [23]:
d = D()
d.pingpong()
d.pingpong()

ping: <__main__.D object at 0x25317a0>
post-ping: <__main__.D object at 0x25317a0>
ping: <__main__.D object at 0x25317a0>
pong: <__main__.D object at 0x25317a0>
pong: <__main__.D object at 0x25317a0>
PONG: <__main__.D object at 0x25317a0>
ping: <__main__.D object at 0x25317a0>
post-ping: <__main__.D object at 0x25317a0>
ping: <__main__.D object at 0x25317a0>
pong: <__main__.D object at 0x25317a0>
pong: <__main__.D object at 0x25317a0>
PONG: <__main__.D object at 0x25317a0>


In [37]:
bool.__mro__

def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

print_mro(bool)

import numbers
print_mro(numbers.Integral)

import io
print_mro(io.BytesIO)
print_mro(io.TextIOWrapper)

bool, int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object


<h2> Coping with multiple inheritance</h2>

<h3>1. Distinguish interface inheritance from implementation inheritance</h3>
• Inheritance of interface: creates a sub-type, implying an “is-a” relationship. <br>
• Inheritance of implementation: avoids code duplication by reuse.

In [40]:
from abc import ABC, abstractmethod

# Interface (what something can do)
class Speaker(ABC):
    @abstractmethod
    def speak(self):
        pass

# Implementation (how something does it)
class EnglishSpeaker:
    def speak(self):
        print("Hello!")

class Dog(Speaker, EnglishSpeaker): # Inherits interface and implementation
    pass

my_dog = Dog()
my_dog.speak() # Inherits the EnglishSpeaker's implementation

<class 'TypeError'>: Can't instantiate abstract class Dog without an implementation for abstract method 'speak'

In [41]:
from abc import ABC, abstractmethod

class Worker(ABC):
    @abstractmethod
    def work(self):
        pass

    @abstractmethod
    def take_break(self):
        pass

In [42]:
class WritesCode:
    def work(self):
        print("Writing code...")

class TypesDocuments:
    def work(self):
        print("Typing documents...")

class EnjoysCoffeeBreaks:
    def take_break(self):
        print("Taking a coffee break.")

class PrefersWalkingBreaks:
    def take_break(self):
        print("Going for a short walk.")

In [44]:
class Programmer(Worker, WritesCode, EnjoysCoffeeBreaks):
    pass

class Secretary(Worker, TypesDocuments, PrefersWalkingBreaks):
    pass

programmer = Programmer()
secretary = Secretary()

print("Programmer:")
programmer.work()       # Inherits implementation from WritesCode
programmer.take_break() # Inherits implementation from EnjoysCoffeeBreaks

print("\nSecretary:")
secretary.work()        # Inherits implementation from TypesDocuments
secretary.take_break()  # Inherits implementation from PrefersWalkingBreaks

<class 'TypeError'>: Can't instantiate abstract class Programmer without an implementation for abstract methods 'take_break', 'work'

<h2> Make interfaces explicit with ABCs</h2>

If a class is meant to define an interface (a set of methods that subclasses should implement), make it an ABC using the abc module. This clearly signals the intended use.

In [31]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

    @abstractmethod
    def perimeter(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)

# You cannot create an instance of Shape directly because it's abstract.
# my_shape = Shape() # This would raise a TypeError

<h3>3. Use mixins for code reuse</h3>
Mixins are classes designed to provide specific methods that can be reused by multiple other classes that might not be related in terms of "is-a" relationships. A mixin shouldn't be instantiated on its own, and a class shouldn't inherit only from a mixin. Each mixin should focus on a small set of related functionalities.

In [32]:
class LoggingMixin:
    def log(self, message):
        print(f"LOG: {self.__class__.__name__}: {message}")

class SerializableMixin:
    def serialize(self):
        return f"<{self.__class__.__name__}: {self.__dict__}>"

class MyClass(LoggingMixin, SerializableMixin):
    def __init__(self, name, value):
        self.name = name
        self.value = value
        self.log("MyClass instance created")

obj = MyClass("example", 10)
obj.log("Doing something important")
print(obj.serialize())

LOG: MyClass: MyClass instance created
LOG: MyClass: Doing something important
<MyClass: {'name': 'example', 'value': 10}>


<h3>4. Make mixins explicit by naming:</h3>
It's a good practice to name mixin classes with a ...Mixin suffix (e.g., LoggingMixin, IterableMixin). This makes it clear that the class is intended for reuse and not as a primary type in the inheritance hierarchy. The text notes that Tkinter doesn't follow this convention.

<h3>5. An ABC may also be a mixin; the reverse is not true:</h3>
An ABC can have concrete (implemented) methods, so it can provide reusable code like a mixin. Additionally, it defines a type. A mixin, on the other hand, is primarily for reuse and doesn't necessarily define a new type in the same way an ABC does. An ABC can be the sole base class of another class, but a mixin usually isn't.

<h3>6. Don’t subclass from more than one concrete class:</h3>
Concrete classes (classes you can create instances of) should ideally inherit from at most one other concrete class. Any other superclasses should be ABCs or mixins. This helps keep the inheritance structure clearer and reduces potential conflicts from multiple implementation inheritances.

In [33]:
class ConcreteA:
    def method_a(self):
        print("Method A from ConcreteA")

class ConcreteB:
    def method_b(self):
        print("Method B from ConcreteB")

class MixinC:
    def method_c(self):
        print("Method C from MixinC")

class MyClass(ConcreteA, MixinC): # Inherits from one concrete and one mixin
    pass

# class ProblematicClass(ConcreteA, ConcreteB): # Inheriting from two concrete classes can be tricky
#     pass

<h3>7. Provide aggregate classes to users:</h3>

If certain combinations of ABCs and mixins are commonly used, create a new class that inherits from all of them. This "aggregate class" provides a convenient way for users to get the desired functionality without having to remember the order of inheritance or all the individual base classes.

In [35]:
class Walker:
    def walk(self):
        print("Walking...")

class Swimmer:
    def swim(self):
        print("Swimming...")

# Aggregate class for something that can both walk and swim
class Amphibious(Walker, Swimmer):
    pass

duck = Amphibious()
duck.walk()
duck.swim()

Walking...
Swimming...


<h3>8. “Favor object composition over class inheritance.”</h3>

This is a general design principle. Instead of inheriting behavior, a class can contain objects of other classes and ask those objects to perform tasks (delegation). This often leads to more flexible and less tightly coupled designs.

In [36]:
class Dog:
    def bark(self):
        print("Woof!")

class FetchingDog(Dog):
    def fetch(self, item):
        print(f"Fetching the {item}!")

my_dog = Dog()
my_fetching_dog = FetchingDog()

my_dog.bark()          # Woof!
# my_dog.fetch("ball") # Error! Dog doesn't have fetch
my_fetching_dog.bark() # Woof! (inherited)
my_fetching_dog.fetch("ball") # Fetching the ball!

Woof!
Woof!
Fetching the ball!


Inheritance way: If you want a toy car that can also fly, you build a new type of toy car, a "FlyingCar," that is a car but with added wings.

Composition way: You keep your regular toy car. Then, you have a separate "Wings" attachment. You can attach the "Wings" to your regular car to make it fly. The car has wings; it's not a fundamentally different type of car. You can also detach the wings when you don't need them.